##Colab setup

In [6]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110377 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [7]:
!mkdir drive
!google-drive-ocamlfuse drive
!ls drive/Deep_learning_project

mkdir: cannot create directory ‘drive’: File exists
 cache.py				       ngrok-stable-linux-amd64.zip.1
 cifar-10-batches-py			       ngrok-stable-linux-amd64.zip.2
 cifar10.py				       ngrok-stable-linux-amd64.zip.3
'CIFAR10 with logistic regression 4.0.ipynb'   ngrok-stable-linux-amd64.zip.4
 CNN_Scratch.ipynb			       ngrok-stable-linux-amd64.zip.5
 data					       ngrok-stable-linux-amd64.zip.6
'Data Exploration and Preprocessing.ipynb'     ngrok-stable-linux-amd64.zip.7
 dataset.py				       __pycache__
 data_utility.py			       Untitled0.ipynb
 download.py				      'VGG_14&16.ipynb'
 drive					       VGG_19.ipynb
 GR5242_Final_Project_Report.ipynb	       vgg_logs
 logs					       vgg_logs14
 model					       vgg_logs16
 ngrok					       vgg_total
 ngrok-stable-linux-amd64.zip


In [8]:
cd /content/drive/Deep_learning_project

/content/drive/Deep_learning_project


##Code initial

In [0]:
import tensorflow as tf
from data_utility import *

iterations      = 200
batch_size      = 250
total_epoch     = 164
weight_decay    = 0.0003
dropout_rate    = 0.5
momentum_rate   = 0.9
log_save_path   = './vgg_logs16'
model_save_path = './model/'


# ========================================================== #
# ├─ bias_variable()
# ├─ conv2d()           With Batch Normalization
# ├─ max_pool()
# └─ global_avg_pool()
# ========================================================== #


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape, dtype=tf.float32 )
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool(input, k_size=1, stride=1, name=None):
    return tf.nn.max_pool(input, ksize=[1, k_size, k_size, 1], strides=[1, stride, stride, 1], padding='SAME',name=name)

def batch_norm(input):
    return tf.contrib.layers.batch_norm(input, decay=0.9, center=True, scale=True, epsilon=1e-3, is_training=train_flag, updates_collections=None)

# ========================================================== #
# ├─ _random_crop() 
# ├─ _random_flip_leftright()
# ├─ data_augmentation()
# ├─ data_preprocessing()
# └─ learning_rate_schedule()
# ========================================================== #

def _random_crop(batch, crop_shape, padding=None):
        oshape = np.shape(batch[0])
        
        if padding:
            oshape = (oshape[0] + 2*padding, oshape[1] + 2*padding)
        new_batch = []
        npad = ((padding, padding), (padding, padding), (0, 0))
        for i in range(len(batch)):
            new_batch.append(batch[i])
            if padding:
                new_batch[i] = np.lib.pad(batch[i], pad_width=npad,
                                          mode='constant', constant_values=0)
            nh = random.randint(0, oshape[0] - crop_shape[0])
            nw = random.randint(0, oshape[1] - crop_shape[1])
            new_batch[i] = new_batch[i][nh:nh + crop_shape[0],
                                        nw:nw + crop_shape[1]]
        return new_batch

def _random_flip_leftright(batch):
        for i in range(len(batch)):
            if bool(random.getrandbits(1)):
                batch[i] = np.fliplr(batch[i])
        return batch

def data_preprocessing(x_train,x_test):

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    x_train[:,:,:,0] = (x_train[:,:,:,0] - np.mean(x_train[:,:,:,0])) / np.std(x_train[:,:,:,0])
    x_train[:,:,:,1] = (x_train[:,:,:,1] - np.mean(x_train[:,:,:,1])) / np.std(x_train[:,:,:,1])
    x_train[:,:,:,2] = (x_train[:,:,:,2] - np.mean(x_train[:,:,:,2])) / np.std(x_train[:,:,:,2])

    x_test[:,:,:,0] = (x_test[:,:,:,0] - np.mean(x_test[:,:,:,0])) / np.std(x_test[:,:,:,0])
    x_test[:,:,:,1] = (x_test[:,:,:,1] - np.mean(x_test[:,:,:,1])) / np.std(x_test[:,:,:,1])
    x_test[:,:,:,2] = (x_test[:,:,:,2] - np.mean(x_test[:,:,:,2])) / np.std(x_test[:,:,:,2])

    return x_train, x_test

def learning_rate_schedule(epoch_num):
      if epoch_num < 81:
        return 0.1
      elif epoch_num < 121:
        return 0.01
      else:
        return 0.001

def data_augmentation(batch):
    batch = _random_flip_leftright(batch)
    batch = _random_crop(batch, [32,32], 4)
    return batch

def run_testing(sess,ep):
    acc = 0.0
    loss = 0.0
    pre_index = 0
    add = 1000
    for it in range(10):
        batch_x = test_x[pre_index:pre_index+add]
        batch_y = test_y[pre_index:pre_index+add]
        pre_index = pre_index + add
        loss_, acc_  = sess.run([cross_entropy,accuracy],feed_dict={x:batch_x, y_:batch_y, keep_prob: 1.0, train_flag: False})
        loss += loss_ / 10.0
        acc += acc_ / 10.0
    summary = tf.Summary(value=[tf.Summary.Value(tag="test_loss", simple_value=loss), 
                            tf.Summary.Value(tag="test_accuracy", simple_value=acc)])
    return acc, loss, summary



##vgg14

In [7]:
# ========================================================== #
# ├─ main()
# Training and Testing 
# Save train/teset loss and acc for visualization
# Save Model in ./model
# ========================================================== #
tf.reset_default_graph() 

if __name__ == '__main__':

    train_x, train_y, test_x, test_y = prepare_data()
    train_x, test_x = data_preprocessing(train_x, test_x)

    # define placeholder x, y_ , keep_prob, learning_rate
    x  = tf.placeholder(tf.float32,[None, image_size, image_size, 3])
    y_ = tf.placeholder(tf.float32, [None, class_num])
    keep_prob = tf.placeholder(tf.float32)
    learning_rate = tf.placeholder(tf.float32)
    train_flag = tf.placeholder(tf.bool)

    # build_network

    W_conv1_1 = tf.get_variable('conv1_1', shape=[3, 3, 3, 64], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv1_1 = bias_variable([64])
    output  = tf.nn.relu( batch_norm(conv2d(x,W_conv1_1) + b_conv1_1))
    
    W_conv1_2 = tf.get_variable('conv1_2', shape=[3, 3, 64, 64], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv1_2 = bias_variable([64])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv1_2) + b_conv1_2))
    output  = max_pool(output, 2, 2, "pool1")

    W_conv2_1 = tf.get_variable('conv2_1', shape=[3, 3, 64, 128], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv2_1 = bias_variable([128])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv2_1) + b_conv2_1))
    

    W_conv2_2 = tf.get_variable('conv2_2', shape=[3, 3, 128, 128], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv2_2 = bias_variable([128])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv2_2) + b_conv2_2))
    output  = max_pool(output, 2, 2, "pool2")

    W_conv3_1 = tf.get_variable('conv3_1', shape=[3, 3, 128, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_1 = bias_variable([256])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv3_1) + b_conv3_1))


    W_conv3_2 = tf.get_variable('conv3_2', shape=[3, 3, 256, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_2 = bias_variable([256])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv3_2) + b_conv3_2))
    

    W_conv3_3 = tf.get_variable('conv3_3', shape=[3, 3, 256, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_3 = bias_variable([256])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv3_3) + b_conv3_3))
    output  = max_pool(output, 2, 2, "pool3")

    W_conv4_1 = tf.get_variable('conv4_1', shape=[3, 3, 256, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_1 = bias_variable([512])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv4_1) + b_conv4_1))


    W_conv4_2 = tf.get_variable('conv4_2', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_2 = bias_variable([512])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv4_2) + b_conv4_2))

    W_conv4_3 = tf.get_variable('conv4_3', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_3 = bias_variable([512])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv4_3) + b_conv4_3))
    output  = max_pool(output, 2, 2,"pool4")

    W_conv5_1 = tf.get_variable('conv5_1', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_1 = bias_variable([512])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv5_1) + b_conv5_1))

    W_conv5_2 = tf.get_variable('conv5_2', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_2 = bias_variable([512])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv5_2) + b_conv5_2))
    
    W_conv5_3 = tf.get_variable('conv5_3', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_3 = bias_variable([512])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv5_3) + b_conv5_3))
    output  = max_pool(output, 2, 2, "pool5")
    

    # output = tf.contrib.layers.flatten(output)
    shape=int(np.prod(output.get_shape()[1:]))
    output = tf.reshape(output,[-1,shape])

    W_fc1 = tf.get_variable('fc1', shape=[shape,4096], initializer=tf.contrib.keras.initializers.he_normal())
    b_fc1 = bias_variable([4096])
    output = tf.nn.relu( batch_norm(tf.matmul(output,W_fc1) + b_fc1) )
    output  = tf.nn.dropout(output,keep_prob)
    
    W_fc = tf.get_variable('fc2', [4096, 10])
    b_fc = bias_variable([10])
    output = tf.nn.relu( batch_norm(tf.matmul(output,W_fc) + b_fc) )
    output  = tf.nn.dropout(output,keep_prob)
    
    # output  = tf.reshape(output,[-1,10])



    # loss function: cross_entropy
    # train_step: training operation
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=output))
    l2 = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
    train_step = tf.train.MomentumOptimizer(learning_rate, momentum_rate,use_nesterov=True).minimize(cross_entropy + l2 * weight_decay)
    
    correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    # initial an saver to save model
    saver = tf.train.Saver()
   
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter(log_save_path,sess.graph)

        # epoch = 164 
        # make sure [bath_size * iteration = data_set_number]

        for ep in range(1,total_epoch+1):
            lr = learning_rate_schedule(ep)
            pre_index = 0
            train_acc = 0.0
            train_loss = 0.0
            start_time = time.time()

            print("\nepoch %d/%d:" %(ep,total_epoch))

            for it in range(1,iterations+1):
                batch_x = train_x[pre_index:pre_index+batch_size]
                batch_y = train_y[pre_index:pre_index+batch_size]

                batch_x = data_augmentation(batch_x)

                _, batch_loss = sess.run([train_step, cross_entropy],feed_dict={x:batch_x, y_:batch_y, keep_prob: dropout_rate, learning_rate: lr, train_flag: True})
                batch_acc = accuracy.eval(feed_dict={x:batch_x, y_:batch_y, keep_prob: 1.0, train_flag: True})

                train_loss += batch_loss
                train_acc  += batch_acc
                pre_index  += batch_size

                if it == iterations:
                    train_loss /= iterations
                    train_acc /= iterations

                    loss_, acc_  = sess.run([cross_entropy,accuracy],feed_dict={x:batch_x, y_:batch_y, keep_prob: 1.0, train_flag: True})
                    train_summary = tf.Summary(value=[tf.Summary.Value(tag="train_loss", simple_value=train_loss), 
                                          tf.Summary.Value(tag="train_accuracy", simple_value=train_acc)])

                    val_acc, val_loss, test_summary = run_testing(sess,ep)

                    summary_writer.add_summary(train_summary, ep)
                    summary_writer.add_summary(test_summary, ep)
                    summary_writer.flush()

                    print("iteration: %d/%d, cost_time: %ds, train_loss: %.4f, train_acc: %.4f, test_loss: %.4f, test_acc: %.4f" %(it, iterations, int(time.time()-start_time), train_loss, train_acc, val_loss, val_acc))
                else:
                    print("iteration: %d/%d, train_loss: %.4f, train_acc: %.4f" %(it, iterations, train_loss / it, train_acc / it) , end='\r')

        save_path = saver.save(sess, model_save_path)
        print("Model saved in file: %s" % save_path)  

======Loading data======
DataSet aready exist!
Loading ./cifar-10-batches-py/data_batch_1 : 10000.
Loading ./cifar-10-batches-py/data_batch_2 : 10000.
Loading ./cifar-10-batches-py/data_batch_3 : 10000.
Loading ./cifar-10-batches-py/data_batch_4 : 10000.
Loading ./cifar-10-batches-py/data_batch_5 : 10000.
Loading ./cifar-10-batches-py/test_batch : 10000.
Train data: (50000, 32, 32, 3) (50000, 10)
Test data : (10000, 32, 32, 3) (10000, 10)
======Load finished======
======Shuffling data======
======Prepare Finished======

epoch 1/164:
iteration: 200/200, cost_time: 75s, train_loss: 2.2411, train_acc: 0.2248, test_loss: 2.0797, test_acc: 0.3192

epoch 2/164:
iteration: 200/200, cost_time: 71s, train_loss: 2.0213, train_acc: 0.4559, test_loss: 1.6975, test_acc: 0.5204

epoch 3/164:
iteration: 200/200, cost_time: 71s, train_loss: 1.8550, train_acc: 0.6068, test_loss: 1.4860, test_acc: 0.6093

epoch 4/164:
iteration: 200/200, cost_time: 71s, train_loss: 1.7514, train_acc: 0.6909, test_loss: 

##vgg16

In [10]:
log_save_path   = './vgg_logs16'
model_save_path = './model/'
# ========================================================== #
# ├─ main()
# Training and Testing 
# Save train/teset loss and acc for visualization
# Save Model in ./model
# ========================================================== #
tf.reset_default_graph() 

if __name__ == '__main__':

    train_x, train_y, test_x, test_y = prepare_data()
    train_x, test_x = data_preprocessing(train_x, test_x)

    # define placeholder x, y_ , keep_prob, learning_rate
    x  = tf.placeholder(tf.float32,[None, image_size, image_size, 3])
    y_ = tf.placeholder(tf.float32, [None, class_num])
    keep_prob = tf.placeholder(tf.float32)
    learning_rate = tf.placeholder(tf.float32)
    train_flag = tf.placeholder(tf.bool)

    # build_network

    W_conv1_1 = tf.get_variable('conv1_1', shape=[3, 3, 3, 64], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv1_1 = bias_variable([64])
    output  = tf.nn.relu( batch_norm(conv2d(x,W_conv1_1) + b_conv1_1))
    
    W_conv1_2 = tf.get_variable('conv1_2', shape=[3, 3, 64, 64], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv1_2 = bias_variable([64])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv1_2) + b_conv1_2))
    output  = max_pool(output, 2, 2, "pool1")

    W_conv2_1 = tf.get_variable('conv2_1', shape=[3, 3, 64, 128], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv2_1 = bias_variable([128])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv2_1) + b_conv2_1))
    

    W_conv2_2 = tf.get_variable('conv2_2', shape=[3, 3, 128, 128], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv2_2 = bias_variable([128])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv2_2) + b_conv2_2))
    output  = max_pool(output, 2, 2, "pool2")

    W_conv3_1 = tf.get_variable('conv3_1', shape=[3, 3, 128, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_1 = bias_variable([256])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv3_1) + b_conv3_1))


    W_conv3_2 = tf.get_variable('conv3_2', shape=[3, 3, 256, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_2 = bias_variable([256])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv3_2) + b_conv3_2))
    

    W_conv3_3 = tf.get_variable('conv3_3', shape=[3, 3, 256, 256], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv3_3 = bias_variable([256])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv3_3) + b_conv3_3))
    output  = max_pool(output, 2, 2, "pool3")

    W_conv4_1 = tf.get_variable('conv4_1', shape=[3, 3, 256, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_1 = bias_variable([512])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv4_1) + b_conv4_1))


    W_conv4_2 = tf.get_variable('conv4_2', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_2 = bias_variable([512])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv4_2) + b_conv4_2))

    W_conv4_3 = tf.get_variable('conv4_3', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv4_3 = bias_variable([512])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv4_3) + b_conv4_3))
    output  = max_pool(output, 2, 2,"pool4")

    W_conv5_1 = tf.get_variable('conv5_1', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_1 = bias_variable([512])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv5_1) + b_conv5_1))

    W_conv5_2 = tf.get_variable('conv5_2', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_2 = bias_variable([512])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv5_2) + b_conv5_2))
    
    W_conv5_3 = tf.get_variable('conv5_3', shape=[3, 3, 512, 512], initializer=tf.contrib.keras.initializers.he_normal())
    b_conv5_3 = bias_variable([512])
    output  = tf.nn.relu( batch_norm(conv2d(output,W_conv5_3) + b_conv5_3))
    output  = max_pool(output, 2, 2, "pool5")
    

    # output = tf.contrib.layers.flatten(output)
    shape=int(np.prod(output.get_shape()[1:]))
    output = tf.reshape(output,[-1,shape])

    W_fc1 = tf.get_variable('fc1', shape=[shape,4096], initializer=tf.contrib.keras.initializers.he_normal())
    b_fc1 = bias_variable([4096])
    output = tf.nn.relu( batch_norm(tf.matmul(output,W_fc1) + b_fc1) )
    output  = tf.nn.dropout(output,keep_prob)
    
    W_fc2 = tf.get_variable('fc2', shape=[4096,4096], initializer=tf.contrib.keras.initializers.he_normal())
    b_fc2 = bias_variable([4096])
    output = tf.nn.relu( batch_norm(tf.matmul(output,W_fc2) + b_fc2) )
    output  = tf.nn.dropout(output,keep_prob)


    W_fc3 = tf.get_variable('fc3', shape=[4096,1000], initializer=tf.contrib.keras.initializers.he_normal())
    b_fc3 = bias_variable([1000])
    output = tf.nn.relu( batch_norm(tf.matmul(output,W_fc3) + b_fc3) )
    output  = tf.nn.dropout(output,keep_prob)
    
    W_fc = tf.get_variable('W_fc', [1000, 10])
    b_fc = tf.get_variable('b_fc', [10])
    output = tf.nn.relu( batch_norm(tf.matmul(output,W_fc) + b_fc) )
    output  = tf.nn.dropout(output,keep_prob)
    # output  = tf.reshape(output,[-1,10])



    # loss function: cross_entropy
    # train_step: training operation
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=output))
    l2 = tf.add_n([tf.nn.l2_loss(var) for var in tf.trainable_variables()])
    train_step = tf.train.MomentumOptimizer(learning_rate, momentum_rate,use_nesterov=True).minimize(cross_entropy + l2 * weight_decay)
    
    correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    # initial an saver to save model
    saver = tf.train.Saver()
   
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        summary_writer = tf.summary.FileWriter(log_save_path,sess.graph)

        # epoch = 164 
        # make sure [bath_size * iteration = data_set_number]

        for ep in range(1,total_epoch+1):
            lr = learning_rate_schedule(ep)
            pre_index = 0
            train_acc = 0.0
            train_loss = 0.0
            start_time = time.time()

            print("\nepoch %d/%d:" %(ep,total_epoch))

            for it in range(1,iterations+1):
                batch_x = train_x[pre_index:pre_index+batch_size]
                batch_y = train_y[pre_index:pre_index+batch_size]

                batch_x = data_augmentation(batch_x)

                _, batch_loss = sess.run([train_step, cross_entropy],feed_dict={x:batch_x, y_:batch_y, keep_prob: dropout_rate, learning_rate: lr, train_flag: True})
                batch_acc = accuracy.eval(feed_dict={x:batch_x, y_:batch_y, keep_prob: 1.0, train_flag: True})

                train_loss += batch_loss
                train_acc  += batch_acc
                pre_index  += batch_size

                if it == iterations:
                    train_loss /= iterations
                    train_acc /= iterations

                    loss_, acc_  = sess.run([cross_entropy,accuracy],feed_dict={x:batch_x, y_:batch_y, keep_prob: 1.0, train_flag: True})
                    train_summary = tf.Summary(value=[tf.Summary.Value(tag="train_loss", simple_value=train_loss), 
                                          tf.Summary.Value(tag="train_accuracy", simple_value=train_acc)])

                    val_acc, val_loss, test_summary = run_testing(sess,ep)

                    summary_writer.add_summary(train_summary, ep)
                    summary_writer.add_summary(test_summary, ep)
                    summary_writer.flush()

                    print("iteration: %d/%d, cost_time: %ds, train_loss: %.4f, train_acc: %.4f, test_loss: %.4f, test_acc: %.4f" %(it, iterations, int(time.time()-start_time), train_loss, train_acc, val_loss, val_acc))
                else:
                    print("iteration: %d/%d, train_loss: %.4f, train_acc: %.4f" %(it, iterations, train_loss / it, train_acc / it) , end='\r')

        save_path = saver.save(sess, model_save_path)
        print("Model saved in file: %s" % save_path)  

======Loading data======
DataSet aready exist!
Loading ./cifar-10-batches-py/data_batch_1 : 10000.
Loading ./cifar-10-batches-py/data_batch_2 : 10000.
Loading ./cifar-10-batches-py/data_batch_3 : 10000.
Loading ./cifar-10-batches-py/data_batch_4 : 10000.
Loading ./cifar-10-batches-py/data_batch_5 : 10000.
Loading ./cifar-10-batches-py/test_batch : 10000.
Train data: (50000, 32, 32, 3) (50000, 10)
Test data : (10000, 32, 32, 3) (10000, 10)
======Load finished======
======Shuffling data======
======Prepare Finished======
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.


epoch 1/164:
iteration: 200/200, cost_time: 89s, train_loss: 2.3090, train_acc: 0.1097, test_loss: 2.3009, test_acc: 0.1124

epoch 2/164:
iteration: 200/200, cost_time: 78s, train_loss: 2.3017, train_acc: 0.1171, test_loss: 2.2983, test_acc: 0.1264

epoch 3/164:
iteration: 200/200, 

##Tensorboard

In [0]:
# Run Tensorflow in the background - note that we specify the log 
# directory we want to look at
# vgg_logs14
LOG_DIR = 'vgg_logs16'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [17]:
# Download and unzip ngrok - you will only need to do this once per session
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

--2018-12-11 23:07:30--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.22.145.207, 52.203.53.176, 52.203.102.189, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.22.145.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.9’

ngrok-stable-linux- 100%[===================>]   5.11M  8.57MB/s    in 0.6s    

2018-12-11 23:07:35 (8.57 MB/s) - ‘ngrok-stable-linux-amd64.zip.9’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
# Launch the ngrok background process
get_ipython().system_raw('./ngrok http 6006 &')

In [19]:
# Get the public URL and be sorted!
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://61f7d0c8.ngrok.io
